This script calculates pairwise distances between alphas and betas in 3 different ways. 

Needs to be run in the python<3.11 environment (environment_distances.yml)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
# from scipy.stats import pearsonr
# from scipy.spatial import distance
# from scipy.cluster import hierarchy

In [2]:
import functions.pwdistances as pw

For TCRdist, we use the following: `pwseqdist`

this is the function in the background of tcrdist3
my understanding is that tcrdist3 iterates over all these components and calculates a dist score for each
then it combines all the dist scores with some weighting
https://github.com/kmayerb/tcrdist3/blob/55d906b19e4c5038f5fdde843eb2edf8293efd88/tcrdist/repertoire.py#L312
https://github.com/kmayerb/tcrdist3/blob/55d906b19e4c5038f5fdde843eb2edf8293efd88/tcrdist/rep_funcs.py#L33
the function in the tcrdist repo is a wrapper to this: 
https://github.com/agartland/pwseqdist/blob/852e159ad12582973bfbf23dd33fef068723742e/pwseqdist/nb_metrics.py#L201
which is (I think) essentially an implementation of the original tcrdist function

In [3]:
def calculate_dists_and_plot(epdf, name):
    dijA_triplet = pw.triplet_diversity(epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist()).round(2)*100
    dijB_triplet = pw.triplet_diversity(epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist()).round(2)*100
    
    dijA_lev = pw.levenshtein_dist(epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist()).round(0)
    dijB_lev = pw.levenshtein_dist(epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist()).round(0)

    dijA_lev_w = pw.weighted_levenshtein_dist(epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist())
    dijB_lev_w = pw.weighted_levenshtein_dist(epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist())

    dijA_tcrdist = pw.tcrdist_cdr3s(epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist())
    dijB_tcrdist = pw.tcrdist_cdr3s(epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist())
    
    # save everything - I need to do this because pw and julia are not compatible with each other
    # (this is because of numba, and it's a well-known issue)

    dijA_lev1 = pd.DataFrame(dijA_lev)
    dijB_lev1 = pd.DataFrame(dijB_lev)
    dijA_lev1.index = epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist()
    dijA_lev1.columns = epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist()
    dijB_lev1.index = epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist()
    dijB_lev1.columns = epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist()
    dijA_lev1.to_csv('data/output/pairwise_distances/cdr3/dijA_lev_' + name + '.csv')
    dijB_lev1.to_csv('data/output/pairwise_distances/cdr3/dijB_lev_' + name + '.csv')

    dijA_lev_w1 = pd.DataFrame(dijA_lev_w)
    dijB_lev_w1 = pd.DataFrame(dijB_lev_w)
    dijA_lev_w1.index = epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist()
    dijA_lev_w1.columns = epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist()
    dijB_lev_w1.index = epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist()
    dijB_lev_w1.columns = epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist()
    dijA_lev_w1.to_csv('data/output/pairwise_distances/cdr3/dijA_weighted_lev_' + name + '.csv')
    dijB_lev_w1.to_csv('data/output/pairwise_distances/cdr3/dijB_weighted_lev_' + name + '.csv')

    dijA_triplet1 = pd.DataFrame(dijA_triplet)
    dijB_triplet1 = pd.DataFrame(dijB_triplet)
    dijA_triplet1.index = epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist()
    dijA_triplet1.columns = epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist()
    dijB_triplet1.index = epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist()
    dijB_triplet1.columns = epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist()
    dijA_triplet1.to_csv('data/output/pairwise_distances/cdr3/dijA_triplet_' + name + '.csv')
    dijB_triplet1.to_csv('data/output/pairwise_distances/cdr3/dijB_triplet_' + name + '.csv')

    dijA_tcrdist1 = pd.DataFrame(dijA_tcrdist)
    dijB_tcrdist1 = pd.DataFrame(dijB_tcrdist)
    dijA_tcrdist1.index = epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist()
    dijA_tcrdist1.columns = epdf['cdr3a_IMGTgaps'].str.replace('-','').tolist()
    dijB_tcrdist1.index = epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist()
    dijB_tcrdist1.columns = epdf['cdr3b_IMGTgaps'].str.replace('-','').tolist()
    dijA_tcrdist1.to_csv('data/output/pairwise_distances/cdr3/dijA_tcrdist_' + name + '.csv')
    dijB_tcrdist1.to_csv('data/output/pairwise_distances/cdr3/dijB_tcrdist_' + name + '.csv')


    # # plot correlation between alphas and betas
    # f, ax = plt.subplots(ncols = 3, figsize = (12,4))

    # A  = np.tril(dijA_triplet, k = -1)
    # B  = np.tril(dijB_triplet, k = -1)
    # ax[0].scatter(A[A!=''].ravel(), B[B!=''].ravel(), alpha = 0.1, s = 5)
    # ax[0].set_xlabel('triplet diversity - alpha')
    # ax[0].set_ylabel('triplet diversity - beta')
    # C = pearsonr(dijA_triplet.ravel(), dijB_triplet.ravel())
    # title = 'Triplet kernel\nPearson correlation: ' + str(round(C[0], 2)) + ', pval: ' + str(round(C[1],4))
    # ax[0].set_title(title)

    # A  = np.tril(dijA_lev, k = -1)
    # B  = np.tril(dijB_lev, k = -1)
    # ax[1].scatter(A[A!=''].ravel(), B[B!=''].ravel(), alpha = 0.1, s = 5)
    # ax[1].set_xlabel('Lev distance - alpha')
    # ax[1].set_ylabel('Lev distance - beta')
    # C = pearsonr(dijA_lev.ravel(), dijB_lev.ravel())
    # title = 'Levenshtein distance\nPearson correlation: ' + str(round(C[0], 2)) + ', pval: ' + str(round(C[1],4))
    # ax[1].set_title(title)

    # A  = np.tril(dijA_tcrdist, k = -1)
    # B  = np.tril(dijB_tcrdist, k = -1)
    # ax[2].scatter(A[A!=''].ravel(), B[B!=''].ravel(), alpha = 0.1, s = 5)
    # ax[2].set_xlabel('tcrdist - alpha')
    # ax[2].set_ylabel('tcrdist - beta')
    # C = pearsonr(dijA_tcrdist.ravel(), dijB_tcrdist.ravel())
    # title = 'TCRdist\nPearson correlation: ' + str(round(C[0], 2)) + ', pval: ' + str(round(C[1],4))
    # ax[2].set_title(title)
    
    # f.suptitle(epdf.Epitope.unique()[0])
    # plt.tight_layout()
    # plt.show()

    # # plot cluster maps
    # f, ax = plt.subplots(ncols = 3, figsize = (12,4))

    # X = np.tril(dijA_triplet, k=0) + np.triu(dijB_triplet, k=0)
    # Z = hierarchy.leaves_list(
    #     hierarchy.linkage(
    #     distance.pdist(dijA_triplet), method='average'))
    # X = X[Z,:]
    # X = X[:,Z]
    # sns.heatmap(X, ax = ax[0])
    # ax[0].set_title('Triplet kernel')

    # X = np.tril(dijA_lev, k=0) + np.triu(dijB_lev, k=0)
    # Z = hierarchy.leaves_list(
    #     hierarchy.linkage(
    #     distance.pdist(dijA_lev), method='average'))
    # X = X[Z,:]
    # X = X[:,Z]
    # sns.heatmap(X, ax = ax[1])
    # ax[1].set_title('Levenshtein')

    # X = np.tril(dijA_tcrdist, k=0) + np.triu(dijB_tcrdist, k=0)
    # Z = hierarchy.leaves_list(
    #     hierarchy.linkage(
    #     distance.pdist(dijA_tcrdist), method='average'))
    # X = X[Z,:]
    # X = X[:,Z]
    # sns.heatmap(X, ax = ax[2])
    # ax[2].set_title('TCRdist')

    # f.suptitle(epdf.Epitope.unique()[0])
    # plt.tight_layout()
    # plt.show()

    # # create networks

#    # these thresholds are 1/10^4 FDR
#     dijA_triplet1 = np.array(dijA_triplet < 24)
#     dijB_triplet1 = np.array(dijB_triplet < 28)
#     # these thresholds are because I believe pC only gets a signal up to 3aa
#     dijA_lev1 = np.array(dijA_lev < 3)
#     dijB_lev1 = np.array(dijB_lev < 3)
#     # 16 is the threshold suggested by tcrdist3 if a hard threshold is to be used
#     # the alternative is to compare to background to set threshold for each TCR
#     dijA_tcrdist1 = np.array(dijA_tcrdist < 16)
#     dijB_tcrdist1 = np.array(dijB_tcrdist < 16)

#     # we set the diagonal to 0 because we do not want to have edges that are self-directed
#     np.fill_diagonal(dijA_triplet1, 0)
#     np.fill_diagonal(dijB_triplet1, 0)
#     np.fill_diagonal(dijA_lev1, 0)
#     np.fill_diagonal(dijB_lev1, 0)
#     np.fill_diagonal(dijA_tcrdist1, 0)
#     np.fill_diagonal(dijB_tcrdist1, 0)

#     gA_triplet = ig.Graph.Adjacency(dijA_triplet1, mode='undirected')
#     gB_triplet = ig.Graph.Adjacency(dijB_triplet1, mode='undirected')
#     gA1 = gA_triplet
#     gA1.vs.select(_degree=0).delete() 
#     gB1 = gB_triplet
#     gB1.vs.select(_degree=0).delete() 

#     gA_lev = ig.Graph.Adjacency(dijA_lev1, mode='undirected')
#     gB_lev = ig.Graph.Adjacency(dijB_lev1, mode='undirected')
#     gA2 = gA_lev
#     gA2.vs.select(_degree=0).delete() 
#     gB2 = gB_lev
#     gB2.vs.select(_degree=0).delete() 

#     gA_tcrdist = ig.Graph.Adjacency(dijA_tcrdist1, mode='undirected')
#     gB_tcrdist = ig.Graph.Adjacency(dijB_tcrdist1, mode='undirected')
#     gA3 = gA_tcrdist
#     gA3.vs.select(_degree=0).delete() 
#     gB3 = gB_tcrdist
#     gB3.vs.select(_degree=0).delete()

    # plot network graphs
    # f, ax = plt.subplots(ncols=3, nrows=2, figsize = (15,8))
    # ax[0,0].set_ylabel('CDR3a', fontsize=16)
    # ax[1,0].set_ylabel('CDR3b', fontsize=16)
    # ax[0,0].set_title('Triplet kernel', fontsize = 16)
    # ax[0,1].set_title('Levenshtein', fontsize = 16)
    # ax[0,2].set_title('TCRdist', fontsize = 16)
    
    # layout = gA1.layout(layout='fruchterman_reingold')
    # ig.plot(gA1, layout=layout, vertex_size=0.7, edge_width=1.5, target=ax[0,0])
    # layout = gB1.layout(layout='fruchterman_reingold')
    # ig.plot(gB1, layout=layout, vertex_size=0.7, edge_width=1.5, target=ax[1,0], vertex_color = 'dodgerblue')

    # layout = gA2.layout(layout='fruchterman_reingold')
    # ig.plot(gA2, layout=layout, vertex_size=0.7, edge_width=1.5, target=ax[0,1])
    # layout = gB2.layout(layout='fruchterman_reingold')
    # ig.plot(gB2, layout=layout, vertex_size=0.7, edge_width=1.5, target=ax[1,1], vertex_color = 'dodgerblue')

    # layout = gA3.layout(layout='fruchterman_reingold')
    # ig.plot(gA3, layout=layout, vertex_size=0.7, edge_width=1.5, target=ax[0,2])
    # layout = gB3.layout(layout='fruchterman_reingold')
    # ig.plot(gB3, layout=layout, vertex_size=0.7, edge_width=1.5, target=ax[1,2], vertex_color = 'dodgerblue')

    # f.suptitle(epdf.Epitope.unique()[0] + '\nNo singlets', fontsize=16)
    # plt.show()
    
    # # compare the different metrics
    # f, ax = plt.subplots(ncols=3, nrows = 2, figsize = (12,8))
    # A1  = np.tril(dijA_triplet, k = -1)
    # A2  = np.tril(dijA_lev, k = -1)
    # A3  = np.tril(dijA_tcrdist, k = -1)

    # ax[0,0].scatter(A2[A1 >= 24].ravel(), A1[A1 >= 24].ravel(), alpha=0.1, s = 5, c = 'b')
    # ax[0,0].scatter(A2[A1 < 24].ravel(), A1[A1 < 24].ravel(), alpha=0.1, s = 5, c = 'r')
    # ax[0,0].set_xlabel('levenshtein')
    # ax[0,0].set_ylabel('triplet kernel')
    # ax[0,0].axvline(2.5, c = 'k', ls = ':')

    # ax[0,1].scatter(A3[A1 >= 24].ravel(), A1[A1 >= 24].ravel(), alpha=0.1, s = 5, c = 'b')
    # ax[0,1].scatter(A3[A1 < 24].ravel(), A1[A1 < 24].ravel(), alpha=0.1, s = 5, c = 'r')
    # ax[0,1].set_xlabel('tcrdist')
    # ax[0,1].set_ylabel('triplet kernel')
    # ax[0,1].axvline(9.9, c = 'k', ls = ':')

    # ax[0,2].scatter(A2[A1 >= 24].ravel(), A3[A1 >= 24].ravel(), alpha=0.1, s = 5, c = 'b')
    # ax[0,2].scatter(A2[A1 < 24].ravel(), A3[A1 < 24].ravel(), alpha=0.1, s = 5, c = 'r')
    # ax[0,2].set_xlabel('levenshtein')
    # ax[0,2].set_ylabel('tcrdist')
    # ax[0,2].axvline(2.5, c = 'k', ls = ':')
    # ax[0,2].axhline(9.9, c = 'k', ls = ':')

    # B1  = np.tril(dijB_triplet, k = -1)
    # B2  = np.tril(dijB_lev, k = -1)
    # B3  = np.tril(dijB_tcrdist, k = -1)

    # ax[1,0].scatter(B2[B1 >= 24].ravel(), B1[B1 >= 24].ravel(), alpha=0.1, s = 5, c = 'b')
    # ax[1,0].scatter(B2[B1 < 24].ravel(), B1[B1 < 24].ravel(), alpha=0.1, s = 5, c = 'r')
    # ax[1,0].set_xlabel('levenshtein')
    # ax[1,0].set_ylabel('triplet kernel')
    # ax[1,0].axvline(2.5, c = 'k', ls = ':')

    # ax[1,1].scatter(B3[B1 >= 24].ravel(), B1[B1 >= 24].ravel(), alpha=0.1, s = 5, c = 'b')
    # ax[1,1].scatter(B3[B1 < 24].ravel(), B1[B1 < 24].ravel(), alpha=0.1, s = 5, c = 'r')
    # ax[1,1].set_xlabel('tcrdist')
    # ax[1,1].set_ylabel('triplet kernel')
    # ax[1,1].set_ylabel('triplet kernel')
    # ax[1,1].axvline(9.9, c = 'k', ls = ':')

    # ax[1,2].scatter(B2[B1 >= 24].ravel(), B3[B1 >= 24].ravel(), alpha=0.1, s = 5, c = 'b')
    # ax[1,2].scatter(B2[B1 < 24].ravel(), B3[B1 < 24].ravel(), alpha=0.1, s = 5, c = 'r')
    # ax[1,2].set_xlabel('levenshtein')
    # ax[1,2].set_ylabel('tcrdist')
    # ax[1,2].axvline(2.5, c = 'k', ls = ':')
    # ax[1,2].axhline(9.9, c = 'k', ls = ':')

    # plt.tight_layout()
    # plt.show()

In [4]:
for f in os.listdir('data/big_epitopes_subsamples/'):
    X = pd.read_csv('data/big_epitopes_subsamples/' + f, index_col=0)
    print(X.shape)
    calculate_dists_and_plot(X, f.strip('.csv'))

(1000, 33)


R[write to console]: Loading required package: kernlab



(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)
(1000, 33)


In [5]:
for f in os.listdir('data/big_epitopes_subsamples_700/'):
    X = pd.read_csv('data/big_epitopes_subsamples_700/' + f, index_col=0)
    print(X.shape)
    calculate_dists_and_plot(X, f.strip('.csv') + '_700')

(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
(700, 33)
